In [1]:
import pandas as pd
import math
import requests
import json
import config

In [47]:
series_url = 'https://download.bls.gov/pub/time.series/ln/ln.series'

In [48]:
df = pd.read_table(series_url)

In [49]:
df1 = df[(df['lfst_code'] == 23) 
   & (df['seasonal'] == 'U') 
   & (df['periodicity_code'] == 'M')
   & (df['ages_code'].isin([20,31,38,42,49,65]))
   & (df['sexs_code'].isin([1,2]))
   & (df['orig_code'].isin([0,1]))
   & (df['race_code'].isin([0,1,3,4]))
        ][['series_id', 'series_title', 'ages_code', 'sexs_code', 'orig_code', 'race_code']]

df1 = df1[~(df1['race_code'] == 0) | ~(df1['orig_code'] == 0)]
df1['series'] = [x.strip(' ') for x in df1['series_id']]
series_dict1 = pd.Series(df1['series_title'].values,index=df1['series'])[:25].to_dict()
series_dict2 = pd.Series(df1['series_title'].values,index=df1['series'])[25:].to_dict()

In [34]:
# Include the start and end year here
date_range = (1993, 2012)

# Divide the date range into BLS-API-friendly length requests
req_no = int(math.ceil((date_range[1] - date_range[0]) / 10.0))
dates = []
for i in range(0,req_no):
    d1 = str(date_range[0]+i*10)
    d2 = str(date_range[0]+i*10+9)
    dates.append((d1,d2))
dates[-1] = (dates[-1][0], str(date_range[1]))

dates

[('1993', '2002'), ('2003', '2012')]

In [60]:
# URL, key, and headers same as above
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}

df3 = pd.DataFrame()  # blank pandas dataframe to be filled later
df4 = pd.DataFrame()  # blank pandas dataframe to be filled later

In [63]:
for sd in [series_dict1, series_dict2]:
    for start, end in dates: 

        # The data sent in the post request now includes a start and end year
        data = json.dumps({"seriesid":sd.keys(), "startyear":start, "endyear":end})
        p = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()
        d = {} # New dictionary to be filled with data
        for series in p['Results']['series']:
            s = series['seriesID']  # Shorten name to 's'

            # Add dictionary entry with series and reverse series order
            d[s] = pd.DataFrame(series['data']).iloc[::-1]
            if len(series['data']) > 0:  # This if/else is to allow for series of different lengths

                # Convert BLS API dates to readable format (YYYY-MM-DD)
                d[s]['date'] = pd.to_datetime(d[s]['period'] + ' ' + d[s]['year'])

                # Keep only date and series values
                d[s] = d[s].set_index('date')['value'].astype(float)

                # Rename and identify values as floating point numbers
                d[s] = d[s].rename(sd[s])
            else:  # If blank, leave as a blank pandas series
                d[s]['date'] = d[s]['value'] = pd.Series()
                d[s] = d[s]['value'].rename(sd[s])

        # Combine the dataframes for each range of years into one by appending        
        if sd == series_dict1:
            df3 = df3.append(pd.concat([d[k] for k in sd.keys()], axis=1))
        else: 
            df4 = df4.append(pd.concat([d[k] for k in sd.keys()], axis=1))

In [66]:
full_data = df3.join(df4)

In [67]:
full_data.to_csv('full_data.csv')

In [73]:
high_tax = pd.Series(full_data.ix['1993-01-01':'2000-12-01'].mean())
low_tax = pd.Series(full_data.ix['2001-01-01':'2012-12-01'].mean())

In [75]:
pd.concat([high_tax, low_tax], axis=1)

,0,1
"(Unadj) Employment-Population Ratio - 55-64 yrs., White Men",65.834375,67.535417
"(Unadj) Employment-Population Ratio - 35-44 yrs., White Men",90.652083,88.439583
"(Unadj) Employment-Population Ratio - 25-34 yrs., White Men",90.381250,86.974306
"(Unadj) Employment-Population Ratio - 65 yrs. & over, White Men",16.496875,19.758333
"(Unadj) Employment-Population Ratio - 35-44 yrs., White Women",74.521875,71.881250
"(Unadj) Employment-Population Ratio - 45-54 yrs., White Women",73.767708,73.452083
(Unadj) Employment-Population Ratio - 45-54 yrs. Hispanic or Latino Women,60.610811,63.497917
(Unadj) Employment-Population Ratio - 35-44 yrs. Hispanic or Latino Women,63.489189,63.570139
(Unadj) Employment-Population Ratio - 25-34 yrs. Hispanic or Latino Women,58.267568,59.045139
"(Unadj) Employment-Population Ratio - 55-64 yrs., White Women",49.023958,56.147222
